In [ ]:
import pandas as pd
from cv2 import cv2
from google.colab import files as FILE
import os
import requests 

train = pd.read_csv("training_ex3_dl2022b.csv")




for i in range(29111):

    img_data = requests.get(train['image'][i]).content
    id = train['id'][i]
    label =  train['label'][i]
    name = str(id) + '_' + str(label) + '.jpg'
    print(name)
    with open(name, 'wb') as handler:
      handler.write(img_data)

    FILE.download(name)


In [ ]:
test = pd.read_csv("test_ex3_dl2022b.csv")

for i in range(7352):
    img_data = requests.get(train['image'][i]).content
    id = train['id'][i]
    name = str(id) + '.jpg'
    print(name)
    with open(name, 'wb') as handler:
      handler.write(img_data)

    FILE.download(name)

In [143]:
from PIL import Image
from os import listdir
from keras.preprocessing.image import img_to_array
import pandas as pd
import numpy as np
import re
import tensorflow
import nltk
import tensorflow as tf
from keras.preprocessing.image import img_to_array
import keras
from keras.models import Sequential
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D
from tensorflow.keras import optimizers
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.layers.pooling import MaxPooling2D
from keras.layers.core import Activation, Flatten
from keras.preprocessing.text import text_to_word_sequence
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout
import numpy as np
import pandas as pd



train_X = []
train_Y = []

imgs = listdir("C:/Users/haimr/OneDrive/שולחן העבודה/train")
for item in imgs: 
    try:
        if item.endswith("jpg") :
          fullname=str(item)
          fullname= fullname.split("_")
          temp=fullname[-1]
          t=temp.replace('.jpg','')
          image = tf.keras.preprocessing.image.load_img("C:/Users/haimr/OneDrive/שולחן העבודה/train/"+item,target_size=(64,64,3))
          reimg = img_to_array(image)
          train_Y.append(t)
          train_X.append(reimg)  
    except:
        pass

train_X = np.array(train_X, dtype="float") / 255.0
train_Y = np.array(train_Y)

In [144]:
transform = {'bergamot': 0,'pomelo': 1,'yuzu': 2,'mandarin': 3,'lemon': 4,'orange': 5,'tangerine': 6}

In [145]:
inverse_transform = {0: 'bergamot',1: 'pomelo',2: 'yuzu',3: 'mandarin',4: 'lemon',5: 'orange',6: 'tangerine'}

In [146]:
for i in range(len(train_Y)):
        train_Y[i]=transform[train_Y[i]]


In [147]:
train_Y = train_Y.astype('uint8')

In [211]:
#Create model
cnn = Sequential()

# Params for model
input_shape=(64, 64, 3)

# Add CNN layers
#  Stack 1:
cnn.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
cnn.add(layers.MaxPooling2D((2, 2)))

# Stack 2:
cnn.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
cnn.add(layers.MaxPooling2D((2, 2)))


cnn.add(Flatten())


# output layer:
cnn.add(layers.Dense(64, activation='relu'))
cnn.add(layers.Dense(7, activation='softmax'))


optimizer='adam'

In [212]:
cnn.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])



In [ ]:
cnn.fit(train_X, train_Y, validation_split=0.2, epochs=10)

Epoch 1/10
368/368 [==============================] - 31s 84ms/step - loss: 1.4579 - accuracy: 0.4370 - val_loss: 1.3155 - val_accuracy: 0.4927
Epoch 2/10
368/368 [==============================] - 32s 87ms/step - loss: 1.2027 - accuracy: 0.5473 - val_loss: 1.2113 - val_accuracy: 0.5386
Epoch 3/10
 97/368 [======>.......................] - ETA: 20s - loss: 1.0838 - accuracy: 0.5905

In [151]:

from PIL import Image
from os import listdir
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
import pandas as pd
import numpy as np
import re
import tensorflow
import nltk
import tensorflow as tf
from keras.preprocessing.image import img_to_array
test_image = []
test_id = []

files = listdir("C:/Users/haimr/OneDrive/שולחן העבודה/test")
for item in files:
    if item.endswith("jpg") :
      temp=str(item)
      t=temp.replace('.jpg','')
      test_id.append(t)
      image2 = tf.keras.preprocessing.image.load_img("C:/Users/haimr/OneDrive/שולחן העבודה/test/"+item,target_size=(64,64,3))
      img_array2 =  img_to_array(image2)
      test_image.append(img_array2)  


test_X = np.array(test_image, dtype="float") / 255.00

In [172]:
haim=pd.read_csv("C:/Users/haimr/OneDrive/שולחן העבודה/sample_submission_ex3_dl2022b.csv")

In [190]:
predict=cnn.predict(test_X)
predict=(predict.argmax(axis=-1)).tolist()
for i in range(len(predict)):
        predict[i]=inverse_transform[predict[i]]

matan=dict(zip(np.array(test_id),predict))

In [192]:
for ids in matan:
    haim.loc[haim['id']==int(ids),'label']=matan[ids]
haim

,id,label
0,1922832217,orange
1,1126456633,bergamot
2,5006892026,bergamot
3,98136071,lemon
4,2579952038,bergamot
...,...,...
7349,3914749590,yuzu
7350,1197098425,lemon
7351,1980553042,orange
7352,3938810201,bergamot


In [140]:
df_submit.to_csv('good.csv', index=False)